VacationPy

Read in file from previous exercise

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import time
from scipy.stats import linregress
from pprint import pprint
import gmaps

# Import API key
from api_keys import g_key

# Incorporated citipy to determine city based on latitude and longitude
from citipy import citipy

In [2]:
city_data = "../WeatherPy/Output/city_weather_data.csv"

city_weather_data = pd.read_csv(city_data, encoding="ISO-8859-1")
city_weather_data.head()

,cityid,cloudiness,country,date_time,humidity,lat,lon,max_temp,name,temp,wind
0,1651810.0,100.0,TL,1.607302e+09,87.0,-8.93,125.41,21.33,airai,21.33,0.85
1,2094027.0,56.0,PG,1.607302e+09,66.0,-6.22,155.63,29.84,kieta,29.84,1.75
2,124291.0,20.0,IR,1.607302e+09,93.0,33.12,46.16,4.00,mehran,4.00,1.00
3,2037078.0,0.0,CN,1.607302e+09,88.0,49.20,119.70,-15.36,hailar,-15.36,6.77
4,6201424.0,78.0,NZ,1.607302e+09,63.0,-46.19,168.86,18.33,mataura,18.33,4.92


Create a heat map that displays the humidity for every city from Part I.

In [3]:
gmaps.configure(api_key=g_key)

locations = city_weather_data[["lat", "lon"]].astype(float)
humidity = city_weather_data["humidity"].astype(float)

In [4]:
fig = gmaps.figure()

heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=100,
                                 point_radius = 1)

fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='420px'))

Narrow down the DataFrame to find your ideal weather condition. For example:


A max temperature lower than 80 degrees but higher than 70.


Wind speed less than 10 mph.


Zero cloudiness.


Drop any rows that don't contain all three conditions. You want to be sure the weather is ideal.

In [5]:
ideal_vacay_cities = city_weather_data[(city_weather_data['max_temp'] > 15)  & (city_weather_data['max_temp'] < 35)  & (city_weather_data['wind']< 5) & (city_weather_data['cloudiness']< 10)] 
ideal_vacay_cities

,cityid,cloudiness,country,date_time,humidity,lat,lon,max_temp,name,temp,wind
17,2206939.0,1.0,NZ,1.607302e+09,63.0,-46.60,168.33,18.33,bluff,18.33,4.92
18,3446866.0,0.0,BR,1.607302e+09,80.0,-11.85,-40.79,18.95,tapiramuta,18.95,0.51
32,1254046.0,0.0,IN,1.607302e+09,80.0,25.52,90.22,15.80,tura,15.80,1.14
36,352628.0,5.0,EG,1.607302e+09,33.0,28.42,30.78,15.55,matay,15.55,4.02
43,2075265.0,0.0,AU,1.607302e+09,66.0,-33.65,115.33,18.33,busselton,17.73,2.68
...,...,...,...,...,...,...,...,...,...,...,...
522,3463124.0,0.0,BR,1.607302e+09,65.0,-11.05,-45.19,22.81,formosa do rio preto,22.81,0.57
526,3671497.0,0.0,CO,1.607302e+09,88.0,10.99,-74.95,27.00,puerto colombia,27.00,3.10
531,186180.0,0.0,KE,1.607302e+09,73.0,3.52,39.06,20.12,moyale,20.12,1.01
536,157429.0,3.0,TZ,1.607302e+09,75.0,-7.91,39.67,26.96,kilindoni,26.96,2.92


Using Google Places API to find the first hotel for each city located within 5000 meters of your coordinates.

In [6]:
location_url = 'https://maps.googleapis.com/maps/api/place/nearbysearch/json'
hotels = pd.DataFrame()

for index, row in ideal_vacay_cities.iterrows():

    target_coordinates = f"{row['lat']},{row['lon']}"
    target_search = "hotel"
    target_radius = 5000
    target_type = "hotel"

    # set up parameters
    params = {
        "location": target_coordinates,
        "keyword": target_search,
        "radius": target_radius,
        "type": target_type,
        "key": g_key
    }

    lodging_response = requests.get(location_url,params=params)
    lodging_json = lodging_response.json()
    if (len(lodging_json['results'])> 0):
        
        hotel_data = pd.Series({"name": lodging_json['results'][0]['name'], 
                                "lat":lodging_json['results'][0]['geometry']['location']['lat'],
                                "lng":lodging_json['results'][0]['geometry']['location']['lng'],
                               "vicinity":lodging_json['results'][0]['vicinity']})
        hotels = hotels.append(hotel_data,ignore_index=True)
print(len(hotels)) 
hotels

56


,lat,lng,name,vicinity
0,-46.612962,168.357609,Land's End Boutique Hotel,"10 Ward Parade, Bluff"
1,-11.848623,-40.791045,Hotel Visão,"Tv. Mariano Alves, Tapiramutá"
2,25.514468,90.192206,Hotel Polo Orchid,"Araimile, Tura"
3,28.423360,30.734995,Nag Aespen,"Madinet Matay, Matay"
4,-33.658383,115.274303,Abbey Beach Resort,"595 Bussell Hwy, Broadwater"
5,-2.582576,150.781292,Nusa Island Retreat,"Nusa Island Kavieng New Ireland PG 631, Kavieng"
6,-33.959609,18.385298,Atlanticview Cape Town Boutique Hotel,"31 Francolin Rd, Camps Bay, Cape Town"
7,-15.531197,-40.908101,Pousada Dona Vitória,"R. Cel. Timóteo Alves, Encruzilhada"
8,-31.248050,-61.490844,Amérian Rafaela Hotel,"Bv. G. Lehmann 501, Rafaela"
9,-15.208865,12.101803,Infotur Namibe,Namibe


Plot the hotels on top of the humidity heatmap with each pin containing the Hotel Name, City, and Country.

In [12]:
locations = hotels[["lat", "lng"]]
names = hotels["name"]

In [13]:
markers = gmaps.marker_layer(locations, info_box_content=[f"{name}" for name in names])
fig.add_layer(markers)
fig


Figure(layout=FigureLayout(height='420px'))